In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, GPUStatsMonitor, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pl_bolts.datamodules.imagenet_datamodule import ImagenetDataModule

from timm.data.transforms_factory import create_transform
from timm.data.mixup import Mixup

from torchvision.models.resnet import resnet50

from vitransformer.lightning import ImageClassifier
from vitransformer.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD, MIXUP_ALPHA

In [2]:
pl.seed_everything(42)

BATCH_SIZE = 64
data_folder = "/home/dima/datasets/imagenet/"

dm = ImagenetDataModule(data_folder, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
dm.train_transforms = create_transform(224, is_training=True, auto_augment='rand-m5-n2')
dm.val_transforms = create_transform(224, is_training=False)
dm.setup()

STEPS_PER_EPOCH = len(dm.train_dataloader())

Global seed set to 42
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


## Train

In [ ]:
NUM_EPOCHS = 50
mixup_fn = Mixup(MIXUP_ALPHA, prob=0.5, num_classes=1000)
model = resnet50()
vit = ImageClassifier(model, lr=1e-4, weight_decay=1e-5, max_iters=NUM_EPOCHS*STEPS_PER_EPOCH)
lr_monitor = LearningRateMonitor(logging_interval='step')
model_checkpoint = ModelCheckpoint(
    monitor='val_loss', 
    filename='resnet50-{epoch}-{val_loss:.3f}', 
    dirpath='/home/dima/ViTransformer/checkpoints/', 
    mode='min',
)
gpu_monitor = GPUStatsMonitor()

trainer = pl.Trainer(
    max_epochs=NUM_EPOCHS,
    gpus=[1],
    logger=TensorBoardLogger('/home/dima/lightning_logs/', name='resnet50_imagenet'),
    callbacks=[lr_monitor, model_checkpoint, gpu_monitor],
    gradient_clip_val=1,
    deterministic=True,
)

trainer.fit(vit, dm)

/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /home/dima/ViTransformer/checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | ResNet           | 25.6 M
1 | train_loss_fn  | CrossEntropyLoss | 0  

Validation sanity check: 0it [00:00, ?it/s]

/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Global seed set to 42


Training: -1it [00:00, ?it/s]

/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument int

...

Validating: 0it [00:00, ?it/s]

/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument int

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Validating: 0it [00:00, ?it/s]

/home/dima/.cache/pypoetry/virtualenvs/vitransformer-feYZhwM1-py3.8/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
